In [1]:
import rosbag2_py
from rclpy.serialization import serialize_message
import std_msgs.msg as std_msgs
import numpy as np
from builtin_interfaces.msg import Time
import ros2_numpy as rnpy

In [2]:
from waymo_open_dataset import v2
import matplotlib.pyplot as plt
import tensorflow as tf
import dask.dataframe as dd

import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-03-27 19:01:18.099124: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# Path to the directory with all components
dataset_dir = '/home/myron/data/Waymo/data'
context = '10504764403039842352_460_000_480_000'

lidar_df = dd.read_parquet(f'{dataset_dir}/lidar/testing_lidar_{context}.parquet')
lidar_calib_df = dd.read_parquet(f'{dataset_dir}/lidar_calibration/testing_lidar_calibration_{context}.parquet')
lidar_pose_df = dd.read_parquet(f'{dataset_dir}/lidar_pose/testing_lidar_pose_{context}.parquet')
# vehicle_pose_df = dd.read_parquet(f'{dataset_dir}/vehicle_pose/testing_vehicle_pose_10504764403039842352_460_000_480_000.parquet')

In [4]:
lidar_lcalib_df = v2.merge(lidar_df, lidar_calib_df, right_group=True)

In [5]:
lidar_lcalib_df.head()

,key.segment_context_name,key.frame_timestamp_micros,key.laser_name,[LiDARComponent].range_image_return1.values,[LiDARComponent].range_image_return1.shape,[LiDARComponent].range_image_return2.values,[LiDARComponent].range_image_return2.shape,[LiDARCalibrationComponent].extrinsic.transform,[LiDARCalibrationComponent].beam_inclination.min,[LiDARCalibrationComponent].beam_inclination.max,[LiDARCalibrationComponent].beam_inclination.values
0,10504764403039842352_460_000_480_000,1558060923374527,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
1,10504764403039842352_460_000_480_000,1558060923474551,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
2,10504764403039842352_460_000_480_000,1558060923574609,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
3,10504764403039842352_460_000_480_000,1558060923674630,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."
4,10504764403039842352_460_000_480_000,1558060923774688,1,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[64, 2650, 4]","[-0.8531144853352591, -0.5217225826529568, 0.0...",-0.315786,0.039599,"[-0.3105449854411111, -0.30006279157785287, -0..."


In [6]:
def tensor_to_pcd_msg(tensor, timestamp: Time, frame_id='/lidar'):
  pc = np.rec.fromarrays(tf.transpose(tf.gather(tensor, [3,4,5,1], axis=1)), dtype=dt)
  return rnpy.array_to_pointcloud2(pc, timestamp, frame_id)

In [7]:
def get_rosbag_options(path, serialization_format='cdr'):
    storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')

    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format=serialization_format,
        output_serialization_format=serialization_format)

    return storage_options, converter_options

In [8]:
bag_writer = rosbag2_py.SequentialWriter()
storage_options, converter_options = get_rosbag_options(path='output/waymo_10504764403039842352_460_000_480_000')
bag_writer.open(storage_options, converter_options)
pcd_topic_info = rosbag2_py._storage.TopicMetadata(
            name='/points2',
            type='sensor_msgs/msg/PointCloud2',
            serialization_format='cdr')
bag_writer.create_topic(pcd_topic_info)

# class RosbagWriter:
#     def __init__(self, topic: str, msg_type: str, name_suffix: str, name_path_prefix = 'output/waymo', serialization='cdr') -> None:
#         self.writer = rosbag2_py.SequentialWriter()
#         storage_options = rosbag2_py._storage.StorageOptions(
#             uri=f'{name_path_prefix}_{name_suffix}',
#             storage_id='sqlite3')
#         converter_options = rosbag2_py._storage.ConverterOptions('', '')
#         self.writer.open(storage_options, converter_options)
#         self.topic_info = rosbag2_py._storage.TopicMetadata(
#             name=topic,
#             type=msg_type,
#             serialization_format=serialization)
#         bag_writer.create_topic(pcd_topic_info)
    
#     def

[INFO] [1679940081.929795937] [rosbag2_storage]: Opened database 'output/waymo_10504764403039842352_460_000_480_000/waymo_10504764403039842352_460_000_480_000_0.db3' for READ_WRITE.


In [9]:
dt = {'names': ('x', 'y', 'z', 'i'), 'formats': (np.float32, np.float32, np.float32, np.float32)}

for i, (_, r) in enumerate(lidar_lcalib_df.iterrows()):
  # Create component dataclasses for the raw data
  lidar = v2.LiDARComponent.from_dict(r)
  timestamp_us = lidar.key.frame_timestamp_micros
  # lidar_pose = v2.LiDARPoseComponent.from_dict(r)
  lidar_calib = v2.LiDARCalibrationComponent.from_dict(r)
  laser_name = int(lidar.key.laser_name)
  # vehicle_pose = v2.VehiclePoseComponent.from_dict(r)

  if(laser_name == 1):
    pc = v2.convert_range_image_to_point_cloud(lidar.range_image_return1, lidar_calib, keep_polar_features=True)
    ts = Time(sec=int(timestamp_us/1e6), nanosec=int((timestamp_us%1e6)*1e3))
    pcd_msg = tensor_to_pcd_msg(pc, ts, '/lidar')
    bag_writer.write('/points2', serialize_message(pcd_msg), int(timestamp_us*1e3))

  # print(pc)
  # print(pc.shape)
  # if(i > 3):
  # break

2023-03-27 19:01:24.113283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1558060923374527
1558060923474551
1558060923574609
1558060923674630
1558060923774688
1558060923874743
1558060923974800
1558060924074821
1558060924174880
1558060924274909
1558060924374971
1558060924474990
1558060924574974
1558060924674992
1558060924774946
1558060924874968
1558060924974953
1558060925074979
1558060925174998
1558060925275021
1558060925375051
1558060925475071
1558060925575058
1558060925675047
1558060925775030
1558060925875055
1558060925975080
1558060926075066
1558060926175080
1558060926275098
1558060926375083
1558060926475067
1558060926575015
1558060926674990
1558060926774979
1558060926874887
1558060926974875
1558060927174768
1558060927474749
1558060927674718
1558060927774705
1558060927874686
1558060927974712
1558060928074739
1558060928174762
1558060928274820
1558060928374842
1558060928474902
1558060928574932
1558060928674990
1558060928775050
1558060928875075
1558060928975131
1558060929075159
1558060929175184
1558060929275209
1558060929375235
1558060929475224
15580609295752

In [10]:
del bag_writer